# FP management

In thi notebook we will define the various methods to perform changes to the database to keep track of the various events. In particular we will take care of:

- Adding new free agent players to the database, when they are included in the FC list
- Assigning trophies
- Assigning fines
- Calculating total roster value to determine the salary, extra budget and luxury tax
- TBD
- UPDATE PLAYER NAME IF IT HAS CHANGED ON FC:
    - Sometime FC changes the name of a player. In 2019/2020 LUKAKU was 'LUKAKU R.', now just 'LUKAKU', because Jordan left serie A. We need a way of scanning the players in our db by ID and update the name as needed. Maybe it can be included in the stats update algorithm
    
Note: We need also to decide what to do when a player changes team within serie a. 

In [8]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
import json
import copy
from pymongo import MongoClient
from pprint import pprint
import pymongo
import progressbar
with open('credential.json','r') as f:
    cred = json.load(f)
stats = pd.read_pickle('../IGNOBEL/Statistiche_giocatori.pkl')

## Update personal info
this part is to run through the players in our database and cross check the info with the stats db:
- if the player is not in our db, we don't care: we'll add it when someone wants to buy them
- if the player is in our db but not in stats it means that it left serie A, so the algorithm should change the team to none:
```collection.update_one({'_id': pl['_id']},{'$set':{'info.personal_info.team_real': None}})
```
- if the player has changed teams we need to update the information

## UPDATE stats

In [9]:
def update_stats(stats, Id, CR = cred['cred'], DB = 'Game', CO = 'Players'):

    cluster = MongoClient(CR)
    db = cluster[DB]
    collection = db[CO]
    stats.index = list(stats.Id)
    #print(stats[stats['Id'] == int(Id)].T[0][4:20])
    stats_dict = dict(stats[stats['Id'] == int(Id)].T[Id][4:20])

    collection.update_one({'_id': Id}, {'$set': {'info.stats': stats_dict}})

In [10]:
def update_stats_all(stats, CR = cred['cred'], DB = 'Game', CO = 'Players'):
    
    cluster = MongoClient(CR)
    db = cluster[DB]
    collection = db[CO]
    
    all_pl = list(collection.find({}))
    
    for pl in progressbar.progressbar(all_pl):
        #print(pl['_id'])
        if len(stats[stats['Id'] == pl['_id']]) == 0:
            print(pl['name']+' left serie A')
            #
            continue
        else:
            update_stats(stats, pl['_id'], CR, DB, CO)

In [5]:
# algorithm runs through the entire db and updates stats value
# if player no more in serie a it prints a message, but we need to include change of team_real to none
#it is probably best to 
update_stats_all(stats)

/Users/lucaagozzino/opt/anaconda3/lib/python3.7/site-packages/pymongo/common.py:559: UserWarning: Unknown option retryWrites
  warnings.warn(str(exc))
 65% (368 of 563) |##############        | Elapsed Time: 0:01:19 ETA:   0:02:07

EMPEREURleft serie a, modify algorithm to change team_real to None. profile not in FC db anymore


100% (563 of 563) |######################| Elapsed Time: 0:01:58 Time:  0:01:58


## Add player to database